In [1]:
# 02_feature_engineering.ipynb
# Purpose: Join daily time-series data with master tables to create a modelling dataset.
# Inputs:  raw/datasets/*.csv
# Outputs: processed/training_table.csv
# Run this notebook BEFORE 03_model_baseline_xgboost.ipynb


In [2]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..")
DATA_ROOT = PROJECT_ROOT / "Shared project assets" / "data" / "raw" / "datasets"
PROC_ROOT = PROJECT_ROOT / "Shared project assets" / "data" / "processed"

PROC_ROOT.mkdir(parents=True, exist_ok=True)


In [3]:
sku = pd.read_csv(DATA_ROOT / "sku_master.csv")
location = pd.read_csv(DATA_ROOT / "location_master.csv")
festival = pd.read_csv(DATA_ROOT / "festival_calendar.csv", parse_dates=["date"])
macro = pd.read_csv(DATA_ROOT / "monthly_macro.csv")

print("SKU columns      :", list(sku.columns))
print("Location columns :", list(location.columns))
print("Festival columns :", list(festival.columns))
print("Macro columns    :", list(macro.columns))


SKU columns      : ['sku_id', 'brand', 'category', 'segment', 'pack_size', 'material', 'base_price', 'cost']
Location columns : ['location_id', 'city', 'region', 'population', 'avg_income_index']
Festival columns : ['date', 'festival']
Macro columns    : ['month', 'gdp_growth', 'cpi_index', 'consumer_confidence']


In [4]:
def add_sku_features(df: pd.DataFrame) -> pd.DataFrame:
    """Merge SKU attributes."""
    return df.merge(sku, on="sku_id", how="left")


def add_location_features(df: pd.DataFrame) -> pd.DataFrame:
    """Merge location attributes."""
    return df.merge(location, on="location_id", how="left")


def add_festival_flags(df: pd.DataFrame) -> pd.DataFrame:
    """Add is_festival flag from festival_calendar."""
    fest = festival[["date"]].copy()
    fest["is_festival"] = 1
    out = df.merge(fest, on="date", how="left")
    out["is_festival"] = out["is_festival"].fillna(0).astype(int)
    return out


def add_macro_features(df: pd.DataFrame) -> pd.DataFrame:
    """Merge monthly macro indicators."""
    tmp = df.copy()
    tmp["month"] = tmp["date"].dt.month
    
    macro_tmp = macro.copy()
    macro_tmp["month"] = pd.to_datetime(macro_tmp["month"]).dt.month
    macro_tmp = macro_tmp.drop_duplicates(subset=["month"])
    
    return tmp.merge(macro_tmp, on="month", how="left")


def add_calendar_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add basic calendar features."""
    tmp = df.copy()
    tmp["day_of_week"] = tmp["date"].dt.weekday
    tmp["week_of_year"] = tmp["date"].dt.isocalendar().week.astype(int)
    tmp["is_weekend"] = tmp["day_of_week"].isin([5, 6]).astype(int)
    return tmp


def add_lag_features(df: pd.DataFrame, target: str = "units_sold") -> pd.DataFrame:
    """Add lag and rolling features per SKU × location."""
    tmp = df.sort_values(["sku_id", "location_id", "date"]).copy()
    group = ["sku_id", "location_id"]
    
    tmp["lag_7"] = tmp.groupby(group)[target].shift(7)
    
    tmp["rolling_14_mean"] = (
        tmp.groupby(group)[target]
        .transform(lambda x: x.shift(1).rolling(window=14, min_periods=7).mean())
    )
    
    return tmp


In [5]:
daily_path = DATA_ROOT / "daily_timeseries.csv"
df_daily = pd.read_csv(daily_path, parse_dates=["date"])

print("Loaded daily_timeseries.csv")
print("Shape:", df_daily.shape)
print("Columns:", list(df_daily.columns))
print("Date range:", df_daily["date"].min(), "→", df_daily["date"].max())
df_daily.head()


Loaded daily_timeseries.csv
Shape: (91000, 6)
Columns: ['date', 'sku_id', 'location_id', 'units_sold', 'price', 'promo_flag']
Date range: 2024-01-01 00:00:00 → 2024-03-31 00:00:00


,date,sku_id,location_id,units_sold,price,promo_flag
0,2024-01-01,SKU_0001,LOC_001,67,12.18,0
1,2024-01-02,SKU_0001,LOC_001,69,8.70,0
2,2024-01-03,SKU_0001,LOC_001,70,12.56,1
3,2024-01-04,SKU_0001,LOC_001,67,13.05,0
4,2024-01-05,SKU_0001,LOC_001,67,8.05,1


In [6]:
df_features = (
    df_daily
    .pipe(add_sku_features)
    .pipe(add_location_features)
    .pipe(add_festival_flags)
    .pipe(add_macro_features)
    .pipe(add_calendar_features)
    .pipe(add_lag_features)
)

# Drop rows with missing lags (early dates)
min_date = df_features["date"].min() + pd.Timedelta(days=21)
df_features = df_features[df_features["date"] >= min_date].dropna(
    subset=["lag_7", "rolling_14_mean"]
)

print("Training table shape:", df_features.shape)
print("Columns:", list(df_features.columns))
df_features.head()


Training table shape: (70000, 27)
Columns: ['date', 'sku_id', 'location_id', 'units_sold', 'price', 'promo_flag', 'brand', 'category', 'segment', 'pack_size', 'material', 'base_price', 'cost', 'city', 'region', 'population', 'avg_income_index', 'is_festival', 'month', 'gdp_growth', 'cpi_index', 'consumer_confidence', 'day_of_week', 'week_of_year', 'is_weekend', 'lag_7', 'rolling_14_mean']


,date,sku_id,location_id,units_sold,price,promo_flag,brand,category,segment,pack_size,...,is_festival,month,gdp_growth,cpi_index,consumer_confidence,day_of_week,week_of_year,is_weekend,lag_7,rolling_14_mean
21,2024-01-22,SKU_0001,LOC_001,59,10.96,0,Brand5,SNACKS,TOOTHPASTE,2L,...,0,1,0.0199,100.58,109.56,0,4,0,66.0,64.500000
22,2024-01-23,SKU_0001,LOC_001,75,13.53,0,Brand5,SNACKS,TOOTHPASTE,2L,...,0,1,0.0199,100.58,109.56,1,4,0,65.0,64.142857
23,2024-01-24,SKU_0001,LOC_001,70,14.48,0,Brand5,SNACKS,TOOTHPASTE,2L,...,0,1,0.0199,100.58,109.56,2,4,0,75.0,64.214286
24,2024-01-25,SKU_0001,LOC_001,64,10.60,0,Brand5,SNACKS,TOOTHPASTE,2L,...,0,1,0.0199,100.58,109.56,3,4,0,70.0,63.642857
25,2024-01-26,SKU_0001,LOC_001,59,12.66,0,Brand5,SNACKS,TOOTHPASTE,2L,...,0,1,0.0199,100.58,109.56,4,4,0,62.0,63.357143


In [7]:
out_path = PROC_ROOT / "training_table.csv"
df_features.to_csv(out_path, index=False)
print("Saved training_table.csv to:", out_path)


Saved training_table.csv to: ..\Shared project assets\data\processed\training_table.csv
